In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage1_binary.train \
  --config configs/stage1_binary.yaml \
  --data_root "$DATA_ROOT" \
  --out_dir runs/stage1_binary

In [ ]:
#CELL 4: Roboflow format to CSV format

import os
import shutil
import pandas as pd

roboflow_root = dataset.location
output_root = "/content/drive/MyDrive/MADE_Unmade/data_stage1"

for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_root, "images", split), exist_ok=True)

def convert_split(split_name, label_map={"Made": 0, "Unmade": 1}):
    rows = []
    target_split = "val" if split_name == "valid" else split_name
    dst_dir = os.path.join(output_root, "images", target_split)

    for class_name, label in label_map.items():
        src_dir = os.path.join(roboflow_root, split_name, class_name)

        if not os.path.exists(src_dir):
            print(f"Warning: {src_dir} not found, skipping...")
            continue

        img_files = [
            f for f in os.listdir(src_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]

        for img_file in img_files:
            shutil.copy2(
                os.path.join(src_dir, img_file),
                os.path.join(dst_dir, img_file)
            )
            rows.append({"image_id": img_file, "label": label})

        print(f"  {class_name}: {len(img_files)} images")

    df = pd.DataFrame(rows)
    csv_name = f"{target_split}.csv"
    df.to_csv(os.path.join(output_root, csv_name), index=False)

    print(f"Created {csv_name} with {len(df)} images\n")
    return df

print("Converting train split...")
train_df = convert_split("train")

print("Converting valid split...")
val_df = convert_split("valid")

print("Converting test split...")
test_df = convert_split("test")

print("\nDataset Summary:")
print(f"  Train: {len(train_df)} images")
print(f"  Val:   {len(val_df)} images")
print(f"  Test:  {len(test_df)} images")